<a href="https://colab.research.google.com/github/anlyons/FPs_R_COOL/blob/main/FPbase_Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('/content/FPbase_251027.csv')
df = df.set_index('name')
df

,agg,doi,genbank,ipg_id,pdb,pdb.0,pdb.1,pdb.10,pdb.2,pdb.3,...,transitions.2.to_state,transitions.2.trans_wave,transitions.3.from_state,transitions.3.to_state,transitions.3.trans_wave,transitions.4.from_state,transitions.4.to_state,transitions.4.trans_wave,uniprot,uuid
name,,,,,,,,,,,,,,,,,,,,,
10B,NaN,10.1126/science.273.5280.1392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RO9XQ
11,NaN,10.1126/science.273.5280.1392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W92N1
22G,t,10.1126/science.1102506,NaN,NaN,NaN,2Z6X,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1UZQ4
(3-F)Tyr-EGFP,NaN,10.1002/cbic.200300818,NaN,NaN,NaN,1RRX,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2VPWT
5B,NaN,10.1126/science.273.5280.1392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G9FG2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoan2RFP,NaN,10.1073/pnas.062552299,AAL23574,1112201,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q8T4U4,HGZ48
zRFP,NaN,10.1007/s12033-008-9131-y,ACD03134,14915008,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B2ZAG2,4PHX7
ZsGreen,t,10.1038/13657,AAF03372,380058,NaN,2OJK,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Q9U6Y5,W8LAG


In [9]:
columns_with_pdb = df.filter(regex='pdb')
print("Columns containing 'pdb':")
print(columns_with_pdb)

Columns containing 'pdb':
               pdb pdb.0 pdb.1 pdb.10 pdb.2 pdb.3 pdb.4 pdb.5 pdb.6 pdb.7  \
name                                                                        
10B            NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
11             NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
22G            NaN  2Z6X   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
(3-F)Tyr-EGFP  NaN  1RRX   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
5B             NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...            ...   ...   ...    ...   ...   ...   ...   ...   ...   ...   
zoan2RFP       NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
zRFP           NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
ZsGreen        NaN  2OJK   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
ZsGreen1       NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   
ZsYellow1      NaN   NaN   NaN    NaN   NaN   NaN 